Instalar liberias adicionales al entorno

In [ ]:
# Si estás en Databricks o un entorno donde ya tienes Spark y dotenv, puedes omitir esta celda
#!pip install python-dotenv

Llamado de librerias necesarias

In [2]:
from pyspark.sql import SparkSession, DataFrame
from eia_api_client import EIAApiClient
import config
from dotenv import load_dotenv
import os

Carga de variables de entorno

In [3]:
# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('API_KEY')

if not api_key:
    raise Exception("API_KEY no encontrada. Revisa el archivo .env")

Crear sesión de spark

In [4]:
spark = SparkSession.builder.appName("PruebaTecnica").getOrCreate()

Llamado al API y carga de la información

In [5]:
client = EIAApiClient(
    api_key=api_key,
    spark_session=spark,
    base_url=config.BASE_URL
)

try:
    df = client.fetch_data_as_spark_df(
        start_year=config.START_YEAR,
        end_year=config.END_YEAR,
        frequency=config.FREQUENCY,
        length=config.LENGTH
    )
    df.printSchema()
except Exception as e:
    print(e)

root
 |-- area-name: string (nullable = true)
 |-- duoarea: string (nullable = true)
 |-- period: string (nullable = true)
 |-- process: string (nullable = true)
 |-- process-name: string (nullable = true)
 |-- product: string (nullable = true)
 |-- product-name: string (nullable = true)
 |-- series: string (nullable = true)
 |-- series-description: string (nullable = true)
 |-- units: string (nullable = true)
 |-- value: string (nullable = true)



Generar archivo

In [6]:
# Definir la ruta completa de forma más clara y segura
export_path = os.path.join(config.FOLDER_DL, config.FILE_DL)

try:
    # Validar que el DataFrame no esté vacío antes de guardar
    if not df.rdd.isEmpty():
        df.write.mode('overwrite').option("header", "true").csv(export_path)
        print(f"✅ Archivo guardado exitosamente en: {export_path}")
    else:
        print("⚠️ El DataFrame está vacío. No se generó el archivo.")
except Exception as e:
    print(f"❌ Error al guardar el archivo: {e}")

✅ Archivo guardado exitosamente en: DL/crude_reserves_prod.csv


Fin de la sesión

In [7]:
spark.stop()